In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Res34SimSiam, Res34SimSiamSplitHeads, Resnet34, Resnet34Baseline, ResBlock
from dataset import Dataset_per_file, Dataset_whole, Dataset_ori, Dataset_valsubset
from torch.utils.data import DataLoader, Subset
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm.auto import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import sys
import os
import matplotlib.pyplot as plt
import gc
import pickle as pkl
from collections import OrderedDict

import pacmap
from annoy import AnnoyIndex

import heartpy as hp
from heartpy.datautils import rolling_mean
from heartpy.peakdetection import detect_peaks
import neurokit2 as nk

import matplotlib

In [9]:
def test_epoch(PPG_model, test_loader, baseline=False, patient_info_path=None):
    with torch.no_grad():
       

        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(tqdm(test_loader, position=0, leave=True)):

#             PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            all_ppg_peaks = []
            tmp = PPG.cpu().numpy()
            for ppg in tqdm(tmp, position=1, leave=False):
            
                PPG_npy = ppg.squeeze()
                rol_mean = rolling_mean(PPG_npy, windowsize = 0.75, sample_rate = 80.0)
                wd = detect_peaks(PPG_npy, rol_mean, ma_perc = 20, sample_rate = 80.0)

                PPG_peaks = np.zeros_like(PPG_npy)
                PPG_peaks[wd['peaklist']] = wd['ybeat']
                # PPG_peaks[wd['peaklist']] = 1
                all_ppg_peaks.append(PPG_peaks.reshape((1, PPG_peaks.shape[0])))
            all_ppg_peaks = np.concatenate(all_ppg_peaks, axis=0)
            PPG_peaks = torch.from_numpy(all_ppg_peaks).float().unsqueeze(1).cuda()
            
            if baseline:
                _, PPG_out = PPG_model(PPG_peaks, PPG_peaks)
            else:
                _, _, _, _, _, PPG_out = PPG_model(PPG_peaks, PPG_peaks)
            
            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()

        precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
        pr_auc = auc(recall, precision)

        print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')
        rounding = 4
        if patient_info_path is not None:
            print(patient_info_path)
            patient_idx_dict = pkl.load(open(patient_info_path, 'rb'))
            unique_patients = list(patient_idx_dict.keys())
            all_aurocs = []
            all_auprcs = []
            for i in tqdm(range(1000)):
                np.random.seed(i)
                random_sample_patients = np.random.choice(unique_patients, len(unique_patients), replace=True)
                sample_idx = []
                random_subsample_idx = []
                for patient in random_sample_patients:
                    random_subsample_idx += patient_idx_dict[patient]
                random_subsample_idx = np.asarray(random_subsample_idx)

                auroc = roc_auc_score(all_targets[random_subsample_idx].detach().cpu().numpy(), PPG_pred_probs[random_subsample_idx].detach().cpu().numpy())
                precision, recall, thresholds = precision_recall_curve(all_targets[random_subsample_idx].detach().cpu().numpy(), PPG_pred_probs[random_subsample_idx].detach().cpu().numpy())
                auprc = auc(recall, precision)

                all_aurocs.append(auroc)
                all_auprcs.append(auprc)

            all_aurocs = np.asarray(all_aurocs)
            all_auprcs = np.asarray(all_auprcs)

            print(f'\t AUROC {round(np.mean(all_aurocs), rounding)} [{round(np.mean(all_aurocs) - 1.96 *  (np.std(all_aurocs) / np.sqrt(len(all_aurocs))) , rounding)} {round(np.mean(all_aurocs) + 1.96 *  (np.std(all_aurocs) / np.sqrt(len(all_aurocs))) , rounding)}]')
            print(f'\t AUPRC {round(np.mean(all_auprcs), rounding)} [{round(np.mean(all_auprcs) - 1.96 *  (np.std(all_auprcs) / np.sqrt(len(all_auprcs))) , rounding)} {round(np.mean(all_auprcs) + 1.96 *  (np.std(all_auprcs) / np.sqrt(len(all_auprcs))) , rounding)}]')


ERROR! Session/line number was not unique in database. History logging moved to new session 309


# SimSiam

In [10]:
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=2500/model_2.pt'
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=2500/model_2.pt'
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt'
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_/model_2.pt'


In [11]:
model = Res34SimSiam(512, 128).cuda()

In [12]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
testing_data_label_pairs = [
#     ['data_simband_ecg_2400.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
#     ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_staford_2400.npy', 'label_staford.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
    
    test_epoch(model, testloader, patient_info_path=pair[2] if len(pair)==3 else None)

    print()

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/857 [00:00<?, ?it/s]

[TEST] 	PPG      F1: 0.5841
[TEST] 	PPG ROC AUC: 0.7761
[TEST] 	PPG PR  AUC: 0.6806
/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl


  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

[TEST] 	PPG      F1: 0.3146
[TEST] 	PPG ROC AUC: 0.6599
[TEST] 	PPG PR  AUC: 0.3931
/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl


  0%|          | 0/1000 [00:00<?, ?it/s]

	 AUROC 0.658 [0.6569 0.6591]
	 AUPRC 0.3888 [0.3847 0.3929]



# SimSiam no predictor

In [14]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/epoch_40_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_False_no_predictor/model_1.pt'

In [15]:
model = Res34SimSiam(512, 128, predictor=False).cuda()

In [16]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [17]:

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for f in os.listdir(test_data_folder):
    if '2400' in f and 'sully' not in f:
        
        x_path = test_data_folder + f
        y_path = test_data_folder + 'label_'+'_'.join(f[:-4].split('_')[1:-1]) + '.npy'

        print(f[:-4])
        print(x_path)
        print(y_path)
        test_dataset = Dataset_ori(x_path, y_path)
        testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

        test_epoch(model, testloader)

        print()

data_simband_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7244
[TEST] 	PPG ROC AUC: 0.9115
[TEST] 	PPG PR  AUC: 0.8661

data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_goodquality.npy
[TEST] 	PPG      F1: 0.9577
[TEST] 	PPG ROC AUC: 0.9996
[TEST] 	PPG PR  AUC: 0.9983

data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_badquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_badquality.npy
[TEST] 	PPG      F1: 0.6944
[TEST] 	PPG ROC AUC: 0.8752
[TEST] 	PPG PR  AUC: 0.8416

data_staford_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_staford_goodquality.npy
[TEST] 	PPG      F1: 0.9805
[TEST] 	PPG ROC AUC: 0.9996
[TEST] 	PPG PR  AUC: 0.998

data_staford_badquality_2400

# SimSiam split heads

In [4]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_splitheads/model_2.pt'

In [5]:
model = Res34SimSiamSplitHeads(512, 128).cuda()

In [6]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_staford_2400.npy', 'label_staford.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

    test_epoch(model, testloader, )

    print()

data_simband_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.5464
[TEST] 	PPG ROC AUC: 0.6293
[TEST] 	PPG PR  AUC: 0.5098

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.8154
[TEST] 	PPG ROC AUC: 0.9243
[TEST] 	PPG PR  AUC: 0.881

data_ucla_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ecg.npy
[TEST] 	PPG      F1: 0.4648
[TEST] 	PPG ROC AUC: 0.7391
[TEST] 	PPG PR  AUC: 0.3608

data_ucla_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.6893
[TEST] 	PPG ROC AUC: 0.9262
[TEST] 	PPG PR  AUC: 0.8028

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDA

# PPG only baseline

In [16]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/ppg_only_baseline/saved_models/epoch_40_ppglr_0.0001_lambda_0.9/PPG_best_2.pt'

In [19]:
model = Resnet34Baseline().cuda()
state_dict = torch.load(MODEL_PATH) 
model.load_state_dict(state_dict)

<All keys matched successfully>

In [22]:

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for f in os.listdir(test_data_folder):
    if '2400' in f:
        
        x_path = test_data_folder + f
        y_path = test_data_folder + 'label_'+'_'.join(f[:-4].split('_')[1:-1]) + '.npy'

        print(f[:-4])
        print(x_path)
        print(y_path)
        test_dataset = Dataset_ori(x_path, y_path)
        testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

        test_epoch(model, testloader, baseline=True)

        print()

data_simband_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG F1: 0.7703488372093024
[TEST] 	PPG AUC: 0.9026940360860827

data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_goodquality.npy
[TEST] 	PPG F1: 0.9189189189189189
[TEST] 	PPG AUC: 0.9997868712702472

data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_badquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_badquality.npy
[TEST] 	PPG F1: 0.7524429967426709
[TEST] 	PPG AUC: 0.8664995622092125

data_staford_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_staford_goodquality.npy
[TEST] 	PPG F1: 0.9202453987730062
[TEST] 	PPG AUC: 0.9988102683204434

data_staford_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_

# Simsiam Adam optim

In [3]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.2_dim1_512_dim2_128_pred_True_bs=5000Adam/model_4.pt'

model = Res34SimSiam(512, 128).cuda()
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy'],
    ['data_staford_2400.npy', 'label_staford.npy'],
    ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy'],
    ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

    test_epoch(model, testloader)

    print()

data_simband_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.6285
[TEST] 	PPG ROC AUC: 0.7267
[TEST] 	PPG PR  AUC: 0.6498

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7578
[TEST] 	PPG ROC AUC: 0.9196
[TEST] 	PPG PR  AUC: 0.8796

data_ucla_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ecg.npy
[TEST] 	PPG      F1: 0.7495
[TEST] 	PPG ROC AUC: 0.904
[TEST] 	PPG PR  AUC: 0.7567

data_ucla_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.714
[TEST] 	PPG ROC AUC: 0.9164
[TEST] 	PPG PR  AUC: 0.7755

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDAT

In [4]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=5000Adam/model_1.pt'

model = Res34SimSiam(512, 128).cuda()
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy'],
    ['data_staford_2400.npy', 'label_staford.npy'],
    ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy'],
    ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

    test_epoch(model, testloader)

    print()

data_simband_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.623
[TEST] 	PPG ROC AUC: 0.6888
[TEST] 	PPG PR  AUC: 0.5651

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7963
[TEST] 	PPG ROC AUC: 0.9155
[TEST] 	PPG PR  AUC: 0.8709

data_ucla_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ecg.npy
[TEST] 	PPG      F1: 0.6999
[TEST] 	PPG ROC AUC: 0.9099
[TEST] 	PPG PR  AUC: 0.733

data_ucla_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.6742
[TEST] 	PPG ROC AUC: 0.9259
[TEST] 	PPG PR  AUC: 0.7779

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDAT

# SGD [VAL] Epoch 1 Loss: -1.3191769726601126 simsam loss : -0.9764100327073032, ce loss : 0.3168215469497266, 	PPG F1: 0.9428633667019257, ECG F1: 0.94055694451495 -- Chose SGD


# Adam [VAL] Epoch 1 Loss: -1.3098863723243863 simsam loss : -0.9926099595630468, ce loss : 0.33766677324062605, 	PPG F1: 0.941941411465695, ECG F1: 0.9302179029918973
